In [1]:
import os
import json
import time
import requests
import random
from collections import OrderedDict
import urllib.request
from tqdm import tqdm
import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain import OpenAI, VectorDBQA
from langchain.chat_models import AzureChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain



from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("env2.env")

def printmd(string):
    display(Markdown(string))
    
#os.makedirs("data/books/",exist_ok=True)
    

BLOB_CONTAINER_NAME = "acccsuite"
BASE_CONTAINER_URL = "https://synpasedlstore.blob.core.windows.net/" + BLOB_CONTAINER_NAME + "/"
#LOCAL_FOLDER = "./data/books/"

MODEL = "gpt-35-turbo-16k" # options: gpt-35-turbo, gpt-35-turbo-16k, gpt-4, gpt-4-32k

#os.makedirs(LOCAL_FOLDER,exist_ok=True)

In [2]:
from dotenv import dotenv_values, load_dotenv
# specify the name of the .env file name 
env_name = ".env2.env" # change to use your own .env file
config = dotenv_values(env_name)
load_dotenv(env_name)

True

In [3]:

# set env variables 
# os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
# os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
# os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
# os.environ["OPENAI_API_TYPE"] = "azure"
# print(dict(config))


In [4]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
import html

def parse_pdf(file, form_recognizer=False, formrecognizer_endpoint=None, formrecognizerkey=None, model="prebuilt-document", from_url=False, verbose=False):
    """Parses PDFs using PyPDF or Azure Document Intelligence SDK (former Azure Form Recognizer)"""
    offset = 0
    page_map = []
    if not form_recognizer:
        if verbose: print(f"Extracting text using PyPDF")
        reader = PdfReader(file)
        pages = reader.pages
        for page_num, p in enumerate(pages):
            page_text = p.extract_text()
            page_map.append((page_num, offset, page_text))
            offset += len(page_text)
    else:
        if verbose: print(f"Extracting text using Azure Document Intelligence")
        credential = AzureKeyCredential(os.environ["FORM_RECOGNIZER_KEY"])
        form_recognizer_client = DocumentAnalysisClient(endpoint=os.environ["FORM_RECOGNIZER_ENDPOINT"], credential=credential)
        
        if not from_url:
            with open(file, "rb") as filename:
                poller = form_recognizer_client.begin_analyze_document(model, document = filename)
        else:
            poller = form_recognizer_client.begin_analyze_document_from_url(model, document_url = file)
            
        form_recognizer_results = poller.result()

        for page_num, page in enumerate(form_recognizer_results.pages):
            tables_on_page = [table for table in form_recognizer_results.tables if table.bounding_regions[0].page_number == page_num + 1]

            # mark all positions of the table spans in the page
            page_offset = page.spans[0].offset
            page_length = page.spans[0].length
            table_chars = [-1]*page_length
            for table_id, table in enumerate(tables_on_page):
                for span in table.spans:
                    # replace all table spans with "table_id" in table_chars array
                    for i in range(span.length):
                        idx = span.offset - page_offset + i
                        if idx >=0 and idx < page_length:
                            table_chars[idx] = table_id

            # build page text by replacing charcters in table spans with table html
            page_text = ""
            added_tables = set()
            for idx, table_id in enumerate(table_chars):
                if table_id == -1:
                    page_text += form_recognizer_results.content[page_offset + idx]
                elif not table_id in added_tables:
                    page_text += table_to_html(tables_on_page[table_id])
                    added_tables.add(table_id)

            page_text += " "
            page_map.append((page_num, offset, page_text))
            offset += len(page_text)

    return page_map

#function to create from pdf to html
def table_to_html(table):
    table_html = "<table>"
    rows = [sorted([cell for cell in table.cells if cell.row_index == i], key=lambda cell: cell.column_index) for i in range(table.row_count)]
    for row_cells in rows:
        table_html += "<tr>"
        for cell in row_cells:
            tag = "th" if (cell.kind == "columnHeader" or cell.kind == "rowHeader") else "td"
            cell_spans = ""
            if cell.column_span > 1: cell_spans += f" colSpan={cell.column_span}"
            if cell.row_span > 1: cell_spans += f" rowSpan={cell.row_span}"
            table_html += f"<{tag}{cell_spans}>{html.escape(cell.content)}</{tag}>"
        table_html +="</tr>"
    table_html += "</table>"
    return table_html

#text to base64 encoding helper function
def text_to_base64(text):
    # Convert text to bytes using UTF-8 encoding
    bytes_data = text.encode('utf-8')

    # Perform Base64 encoding
    base64_encoded = base64.b64encode(bytes_data)

    # Convert the result back to a UTF-8 string representation
    base64_text = base64_encoded.decode('utf-8')

    return base64_text

In [5]:
profile_index_name = "carpediemsearchindex"

### Create Azure Search Vector-based Index
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

In [6]:
#not needed mostly TODO: we'll see this later
"""
index_payload = {
    "@odata.context": "https://hackcsi.search.windows.net/$metadata#indexes/$entity",
    "@odata.etag": "\"0x8DBF02FAD913FA2\"",
    "name": "hackchat",
    "fields": [
      {
        "name": "content",
        "type": "Edm.String",
        "searchable": "true",
        "filterable": "false",
        "retrievable": "true",
        "sortable": "false",
        "facetable": "false",
        "key": "false",
        "synonymMaps": []
      },
      {
        "name": "filepath",
        "type": "Edm.String",
        "searchable": "false",
        "filterable": "false",
        "retrievable": "true",
        "sortable": "false",
        "facetable": "false",
        "key": "false",
        "synonymMaps": []
      },
      {
        "name": "title",
        "type": "Edm.String",
        "searchable": "true",
        "filterable": "false",
        "retrievable": "true",
        "sortable": "false",
        "facetable": "false",
        "key": "false",
        "synonymMaps": []
      },
      {
        "name": "url",
        "type": "Edm.String",
        "searchable": "false",
        "filterable": "false",
        "retrievable": "true",
        "sortable": "false",
        "facetable": "false",
        "key": "false",
        "synonymMaps": []
      },
      {
        "name": "id",
        "type": "Edm.String",
        "searchable": "false",
        "filterable": "true",
        "retrievable": "true",
        "sortable": "true",
        "facetable": "false",
        "key": "true",
        "synonymMaps": []
      },
      {
        "name": "chunk_id",
        "type": "Edm.String",
        "searchable": "false",
        "filterable": "false",
        "retrievable": "true",
        "sortable": "false",
        "facetable": "false",
        "key": "false",
        "synonymMaps": []
      },
      {
        "name": "last_updated",
        "type": "Edm.String",
        "searchable": "false",
        "filterable": "false",
        "retrievable": "true",
        "sortable": "false",
        "facetable": "false",
        "key": "false",
        "synonymMaps": []
      }
    ],
    "scoringProfiles": [],
    "corsOptions": {
      "allowedOrigins": [
        "*"
      ],
      "maxAgeInSeconds": 300
    },
    "semantic": {
      "configurations": [
        {
          "name": "default",
          "prioritizedFields": {
            "titleField": {
              "fieldName": "title"
            },
            "prioritizedContentFields": [
              {
                "fieldName": "content"
              }
            ],
            "prioritizedKeywordsFields": []
          }
        }
      ]
    },
  }
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + profile_index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)
print(r.json())
"""

'\nindex_payload = {\n    "@odata.context": "https://hackcsi.search.windows.net/$metadata#indexes/$entity",\n    "@odata.etag": ""0x8DBF02FAD913FA2"",\n    "name": "hackchat",\n    "fields": [\n      {\n        "name": "content",\n        "type": "Edm.String",\n        "searchable": "true",\n        "filterable": "false",\n        "retrievable": "true",\n        "sortable": "false",\n        "facetable": "false",\n        "key": "false",\n        "synonymMaps": []\n      },\n      {\n        "name": "filepath",\n        "type": "Edm.String",\n        "searchable": "false",\n        "filterable": "false",\n        "retrievable": "true",\n        "sortable": "false",\n        "facetable": "false",\n        "key": "false",\n        "synonymMaps": []\n      },\n      {\n        "name": "title",\n        "type": "Edm.String",\n        "searchable": "true",\n        "filterable": "false",\n        "retrievable": "true",\n        "sortable": "false",\n        "facetable": "false",\n        "k

In [7]:
# from typing import Container
# #from azure.cosmosdb.table.tableservice import TableService,ListGenerator
# from azure.storage.blob import BlobClient, BlobServiceClient, ContainerClient
# from azure.storage.blob import ResourceTypes, AccountSasPermissions
# from azure.storage.blob import generate_account_sas    
# from datetime import *
# import shutil

# #debugging 
# import traceback

# today = str(datetime.now().date())
# # print(today)

# #================================ SOURCE ===============================
# # Source Client
# # connection_string = 'xxxxxxxxxxxxxxxxxxxxxx' # The connection string for the source container
# # account_key = 'xxxxxxxxxxxxxxxxx' # The account key for the source container
# source_container_name = 'manuals' # Name of container which has blob to be copied
# connection_string = config["DOCUMENT_AZURE_STORAGE_CONNECTION_STRING"]
# account_key= config["DOCUMENT_AZURE_STORAGE_CONNECTION_KEY"]

# # Create client
# client = BlobServiceClient.from_connection_string(connection_string) 

# client = BlobServiceClient.from_connection_string(connection_string)
# all_containers = client.list_containers(include_metadata=True)
# # print(list(all_containers))

# for container in all_containers:
#     # print("==========================")
#     print(container['name'], container['metadata'])
#     if container['name'] == 'chatbottrainingdata':
        
#         container_client = client.get_container_client(container.name)
#         blobs_list = container_client.list_blobs()
        
#         try:
#             for blob in blobs_list:
#                 target_blob_name = blob.name
#                 blob_client = client.get_blob_client(container=container['name'], blob=blob.name)
#                 with open(file=os.path.join(r'', blob.name), mode="wb+") as sample_blob:
#                     download_stream = blob_client.download_blob(max_concurrency=2)
#                     destfile = os.path.join(r'data/profiles', blob.name)
#                     sample_blob.write(download_stream.readall())
#                     shutil.move(blob.name, destfile)
#                     with open(destfile, "w") as file:
#                         file.write(str(download_stream.readall()))
#                         file.close()
#         except Exception as e:
#             traceback.print_exc()
    


In [8]:
import os
# assign directory
directory = 'data/profiles'
filepdfs= []
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        #print(f)
        filepdfs.append(f)

In [9]:
#processing pdfs and splitting pages
import time 
pdf_pages_map = dict()
for pdf in filepdfs:
    print("Extracting Text from",pdf,"...")
    
    if not '.amlignore' in pdf:

        # Capture the start time
        start_time = time.time()
        
        # Parse the PDF
        pdf_path = pdf
        pdf_map = parse_pdf(file=pdf_path, form_recognizer=True, verbose=True)
        pdf_pages_map[pdf]= pdf_map
        
        # Capture the end time and Calculate the elapsed time
        end_time = time.time()
        elapsed_time = end_time - start_time

        print(f"Parsing took: {elapsed_time:.6f} seconds")
        print(f"{pdf} contained {len(pdf_map)} pages\n")

Extracting Text from data/profiles/Operation_Maintenance_Manual.pdf ...
Extracting text using Azure Document Intelligence
Parsing took: 10.396366 seconds
data/profiles/Operation_Maintenance_Manual.pdf contained 2 pages

Extracting Text from data/profiles/CSI_Phase_1_rev_3.pdf ...
Extracting text using Azure Document Intelligence
Parsing took: 6.180190 seconds
data/profiles/CSI_Phase_1_rev_3.pdf contained 2 pages

Extracting Text from data/profiles/Operations_Manual_Compressed.pdf ...
Extracting text using Azure Document Intelligence
Parsing took: 6.481581 seconds
data/profiles/Operations_Manual_Compressed.pdf contained 2 pages

Extracting Text from data/profiles/FANUC_Rockwell_Miller_Catalog_Apr_26 _2023_compressed.pdf ...
Extracting text using Azure Document Intelligence
Parsing took: 17.150576 seconds
data/profiles/FANUC_Rockwell_Miller_Catalog_Apr_26 _2023_compressed.pdf contained 2 pages



In [10]:
print(f"Total contained {len(pdf_pages_map)} pages\n")


Total contained 4 pages



In [11]:
from openai import AzureOpenAI

# openai.api_type = "azure"
# openai.api_key = os.environ["AZURE_OPENAI_API_KEY"]
# openai.api_base = os.environ["AZURE_OPENAI_ENDPOINT"]
# openai.api_version = "2023-05-15"
client = AzureOpenAI(
    # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#rest-api-versioning
    api_version="2023-07-01-preview",
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
)



In [12]:
def getmebedding(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    print(list(list(dict(response)['data'][0]))[0][1])
    return list(list(dict(response)['data'][0]))[0][1]

In [13]:
for pdfname,pdfmap in pdf_pages_map.items():
    print("Uploading chunks from",pdfname)
    for page in tqdm(pdfmap):
        try:
            page_num = page[0] + 1
            content = page[2]
            #print('Content::', content)
            head_tail = os.path.split(pdfname)
            book_url = BASE_CONTAINER_URL + head_tail[1]
            upload_payload = {
                "value": [
                    {
                        "id": text_to_base64(pdfname + str(page_num)),
                        "title": f"{pdfname}_page_{str(page_num)}",
                        "chunk": content,
                        "chunkVector": getmebedding(content if content!="" else "-------"),
                        "name": pdfname,
                        "location": book_url,
                        "page_num": page_num,
                        "@search.action": "upload"
                    },
                ]
            }

            r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + profile_index_name + "/docs/index",
                                 data=json.dumps(upload_payload), headers=headers, params=params)
            if r.status_code != 200:
                print(r.status_code)
                print(r.text)
        except Exception as e:
            print("Exception:",e)
            print(content)
            continue

Uploading chunks from data/profiles/Operation_Maintenance_Manual.pdf


100%|██████████| 2/2 [00:00<00:00, 16677.15it/s]


Exception: name 'base64' is not defined
OPERATION AND MAINTENANCE MANUAL LIQUID DELIEVERY SYSTEM FOR THE VIAL FILLING CELL EDUCATIONAL PLATFORM
PREPARED FOR:
CONNECTED SYSTEMS INSTITUTE (CSI) UNIVERSITY OF WISCONSIN - MILWAUKEE 2311 E. Hartford Ave Milwaukee, WI 53211
SYMBIONT PROJECT NO. DB190928
PROJECT COMPLETION DATE: OCTOBER 2020 
Exception: name 'base64' is not defined
GENERAL INFORMATION LIQUID DELIEVERY SYSTEM FOR THE VIAL FILLING CELL EDUCATIONAL PLATFORM CSI AT THE UNIVERSITY OF WISCONSIN - MILWAUKEE MILWAUKEE, WI
<table><tr><td>FACILITY:</td><td>Connected Systems Institute (CSI) University of Wisconsin - Milwaukee 2311 E. Harford Ave. Milwaukee, WI 53211</td></tr><tr><td>CONTACT:</td><td>Charles Mosley</td></tr><tr><td rowSpan=4>CONTRACTOR:</td><td>Symbiont</td></tr><tr><td>6737 W. Washington Street</td></tr><tr><td>Suite 3440</td></tr><tr><td>Milwaukee, WI 53214</td></tr><tr><td>CONTACTS:</td><td>Chris Lemche</td></tr><tr><td rowSpan=2>TELEPHONE:</td><td>T: 414-755-1118</td

100%|██████████| 2/2 [00:00<00:00, 33961.98it/s]


Exception: name 'base64' is not defined
UNIVERSITY of WISCONSIN UWMILWAUKEE
CONNECTED SYSTEMS INSTITUTE
Rockwell Automation
H HASKELL
Functional
Requirements Document FOR
Vial Filling Cell 
Exception: name 'base64' is not defined
HASKELL
UNIVERSITY of WISCONSIN UWMILWAUKEE
CONNECTED SYSTEMS
INSTITUTE
Rockwell Automation
Functional Requirement Specification
Document Authorizations
<table><tr><th></th><th>Name</th><th>Date</th><th>Signature</th></tr><tr><td>Author:</td><td>Alan Diehl</td><td>05/23/2018</td><td></td></tr><tr><td>Reviewer #1:</td><td>Joseph Zaccaria</td><td>05/23/2018</td><td></td></tr><tr><td>Reviewer #2:</td><td></td><td></td><td></td></tr><tr><td>Approver #1:</td><td></td><td></td><td></td></tr><tr><td>Approver #2:</td><td></td><td></td><td></td></tr></table>
Version Date: 2018-09-18
Version: 2
C 2018 Rockwell Automation, Inc. All Rights Reserved
Page i of viii 
Uploading chunks from data/profiles/Operations_Manual_Compressed.pdf


100%|██████████| 2/2 [00:00<00:00, 39945.75it/s]


Exception: name 'base64' is not defined
APT MANUFACTURINGsolutions
ROBOT
CNC | ¡RVISION
FANUC
RTIFIE
ATION
CA
Operations Manual UWM CSI - Vial Fill Test Bed Connected Smart Manufacturing (CSM) APT Manufacturing Solutions
September 16, 2020 Version 1.0 
Exception: name 'base64' is not defined
CSM Operations Manual
Table of Contents
<table><tr><th>1 Introduction</th><th>.. 6</th></tr><tr><td>1.1 ..... Scope and Purpose</td><td>6</td></tr><tr><td>1.2 .. Process Overview</td><td>7</td></tr><tr><td>1.2.1 Station1 - Load / Unload</td><td>7</td></tr><tr><td>1.2.2 Station2 - Dry Fill</td><td>8</td></tr><tr><td>1.2.3 Station3 - Wet Fill (Single Nozzle Sequential).</td><td>9</td></tr><tr><td>1.2.4 Station4 - Wet Fill (Single Nozzle Mixed)</td><td>10</td></tr><tr><td>1.2.5 Station5 - Wet Fill (Individual)</td><td>11</td></tr><tr><td>1.2.6 Station6 - Capping</td><td>12</td></tr><tr><td>1.2.7 Station7 - Vision Inspection</td><td>13</td></tr><tr><td>1.2.8 Station8 - Weigh Check</td><td>14</td></tr><

100%|██████████| 2/2 [00:00<00:00, 32640.50it/s]

Exception: name 'base64' is not defined
USE OUR ONLINE TOOL TO NAVIGATE EQUIPMENT AND PROGRAMS :unselected: :unselected:
https://aptmfg.com/products/program-overview/
FANUC Robotics Courses
FANUC CNC Courses
Rockwell Automation Courses
Miller Welding Courses
APT Integration Courses
Industry Recognized Certifications
<table><tr><th>Learning Level</th><th>Career Path</th><th>Description</th><th>Cert. Type</th><th>ROBO- DRILL</th><th>CERT Cart</th><th>MTEC- SIM</th><th>MTEC</th><th>Weld CERT Cart</th><th>(PLC/HMI) iCC</th><th>AM-CERT</th><th>CSM</th><th>¿IM5.0</th></tr><tr><td rowSpan=16>Level 1</td><td rowSpan=4>F FANUC Robot F Operator - Material Handling - -</td><td>FANUC: HandlingTool Operation and Programming</td><td></td><td>:unselected:</td><td>:selected:</td><td>:selected:</td><td>:selected:</td><td>:unselected:</td><td>/*
:selected:</td><td>:selected:</td><td>:selected:</td><td>:selected:</td></tr><tr><td>FANUC: HandlingPRO</td><td></td><td>:unselected:</td><td>:selected:</td><td

In [14]:

def get_search_results(query: str, indexes: list, 
                       k: int = 5,
                       reranker_threshold: int = 1,
                       sas_token: str = "",
                       vector_search: bool = False,
                       similarity_k: int = 3, 
                       query_vector: list = []):
    
    headers = {'Content-Type': 'application/json','api-key': os.environ["AZURE_SEARCH_KEY"]}
    params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

    agg_search_results = dict()
    
    for index in indexes:
        search_payload = {
            "search": query,
            "queryType": "semantic",
            "semanticConfiguration": "default",
            "count": "true",
            "speller": "lexicon",
            "queryLanguage": "en-us",
            "captions": "extractive",
            "answers": "extractive",
            "top": k
        }
        if vector_search:
            search_payload["vectorQueries"]= [{"kind":"vector", "vector": query_vector, "fields": "contentVector","k": k, "exhaustive": "true"}]
            # search_payload["select"]= "id, title, chunk, name, location"
            search_payload["select"] =  str("id, title, content, chunk_id, url, filepath")
        else:
            #TODO: Fix this afterwards
            search_payload["select"] = str("id, title, chunks, language, name, location, vectorized")
        
        print(search_payload)
        print(json.dumps(search_payload))
        resp = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index + "/docs/search",
                         data=json.dumps(search_payload), headers=headers, params=params)

        search_results = resp.json()
        print('search results: ', search_results)
        agg_search_results[index] = search_results
    
    content = dict()
    ordered_content = OrderedDict()
    
    for index,search_results in agg_search_results.items():
        for result in search_results['value']:
            if result['@search.rerankerScore'] > reranker_threshold: # Show results that are at least N% of the max possible score=4
                content[result['id']]={
                                        "title": result['title'], 
                                        "name": result['filepath'], 
                                        "location": result['url'] + sas_token if result['url'] else "",
                                        "caption": result['@search.captions'][0]['text'],
                                        "index": index,
                                        "content": result["content"],
                                        "chunk_id": result['chunk_id']
                                    }
                if vector_search:
                    content[result['id']]["content"]= result['content']
                    content[result['id']]["score"]= result['@search.score'] # Uses the Hybrid RRF score
              
                else:
                    # content[result['id']]["chunks"]= result['chunks']
                    content[result['id']]["language"]= result['language']
                    content[result['id']]["score"]= result['@search.rerankerScore'] # Uses the reranker score
                    content[result['id']]["vectorized"]= result['vectorized']
                
    # After results have been filtered, sort and add the top k to the ordered_content
    if vector_search:
        topk = similarity_k
    else:
        topk = k*len(indexes)
        
    count = 0  # To keep track of the number of results added
    for id in sorted(content, key=lambda x: content[x]["score"], reverse=True):
        ordered_content[id] = content[id]
        count += 1
        if count >= topk:  # Stop after adding 5 results
            break

    return ordered_content

In [34]:
# QUESTION = "How does the entire manufacturing proces work?"
QUESTION = "What type of robots do you have in the manufacturing process? and what do they do?"


In [35]:
vector_indexes = [profile_index_name]

ordered_results = get_search_results(QUESTION, vector_indexes, 
                                        k=10,
                                        reranker_threshold=1,
                                        vector_search=True, 
                                        similarity_k=10,
                                        query_vector = getmebedding(QUESTION)
                                        )

[0.0004681247810367495, -0.02267768420279026, 6.977249722694978e-05, -0.00874914601445198, -0.010196520015597343, 0.011494613252580166, -0.03112826868891716, 0.01803051121532917, -0.032348476350307465, 0.0035502843093127012, -0.0032290061935782433, -0.0008575526298955083, -0.014512679539620876, -0.0037969218101352453, 0.006062094122171402, 0.028272464871406555, 0.00669166911393404, -0.0011155486572533846, 0.018432920798659325, -0.0002478546230122447, -0.002072080969810486, -0.0007804784108884633, -0.0013849028619006276, -0.025312812998890877, -0.0059128133580088615, -0.01853676699101925, 0.009923920966684818, -0.007963799871504307, 0.02428731881082058, -0.012585011310875416, 0.0321667417883873, -0.008106590248644352, -0.011481632478535175, -0.004897055681794882, -0.02842823602259159, 0.0038228838238865137, 0.01719973236322403, -0.0007086776313371956, 0.030115757137537003, -0.004595248959958553, 0.04099377617239952, 0.034010034054517746, 0.004238273482769728, 0.00763278640806675, -0.040

In [36]:
COMPLETION_TOKENS = 1000
llm = AzureChatOpenAI(deployment_name="gpt-35-turbo-16k", temperature=0.5, max_tokens=COMPLETION_TOKENS, validate_base_url=False, openai_api_version="2023-05-15")


In [37]:
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x167d8ea90> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x167d7bc50> temperature=0.5 openai_api_key='3f478ede0fb043d48fae48e82a3f1b01' openai_proxy='' max_tokens=1000 azure_endpoint='https://open-ai-pop-in.openai.azure.com/' deployment_name='gpt-35-turbo-16k' openai_api_version='2023-05-15' openai_api_type='azure' validate_base_url=False


In [38]:

top_docs = []
for key,value in ordered_results.items():
    location = value["location"] if value["location"] is not None else ""
    # top_docs.append(Document(page_content=value["chunk"], metadata={"source": location+os.environ['BLOB_SAS_TOKEN']}))
    top_docs.append(Document(page_content=value["content"], metadata={"source": location}))

        
print("Number of chunks:",len(top_docs))

Number of chunks: 6


In [39]:
import tiktoken
# Returns the num of tokens used on a string
def num_tokens_from_string(string: str) -> int:
    encoding_name ='cl100k_base'
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Returning the toekn limit based on model selection
def model_tokens_limit(model: str) -> int:
    """Returns the number of tokens limits in a text model."""
    if model == "gpt-35-turbo":
        token_limit = 4096
    elif model == "gpt-4":
        token_limit = 8192
    elif model == "gpt-35-turbo-16k":
        token_limit = 16384
    elif model == "gpt-4-32k":
        token_limit = 32768
    else:
        token_limit = 4096
    return token_limit

# Returns num of toknes used on a list of Documents objects
def num_tokens_from_docs(docs) -> int:
    num_tokens = 0
    for i in range(len(docs)):
        num_tokens += num_tokens_from_string(docs[i].page_content)
    return num_tokens

In [40]:
from langchain.prompts import PromptTemplate


COMBINE_QUESTION_PROMPT_TEMPLATE = """Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text in {language}.
{context}
Question: {question}
Relevant text, if any, in {language}:"""

COMBINE_QUESTION_PROMPT = PromptTemplate(
    template=COMBINE_QUESTION_PROMPT_TEMPLATE, input_variables=["context", "question", "language"]
)


COMBINE_PROMPT_TEMPLATE = """
# Instructions:
- Given the following extracted parts from one or multiple documents, and a question, create a final answer with references. 
- You can only provide numerical references to documents, using this html format: `<sup><a href="url?query_parameters" target="_blank">[number]</a></sup>`.
- The reference must be from the `Source:` section of the extracted part. You are not to make a reference from the content, only from the `Source:` of the extract parts.
- Reference (source) document's url can include query parameters, for example: "https://example.com/search?query=apple&category=fruits&sort=asc&page=1". On these cases, **you must** include que query references on the document url, using this html format: <sup><a href="url?query_parameters" target="_blank">[number]</a></sup>.
- **You can only answer the question from information contained in the extracted parts below**, DO NOT use your prior knowledge.
- Never provide an answer without references.
- If you don't know the answer, just say that you don't know. Don't try to make up an answer.
- Respond in {language}.

=========
QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER IN {language}:"""


COMBINE_PROMPT = PromptTemplate(
    template=COMBINE_PROMPT_TEMPLATE, input_variables=["summaries", "question", "language"]
)

In [41]:
# Calculate number of tokens of our docs
if(len(top_docs)>0):
    tokens_limit = model_tokens_limit(MODEL) # this is a custom function we created in common/utils.py
    prompt_tokens = num_tokens_from_string(COMBINE_PROMPT_TEMPLATE) # this is a custom function we created in common/utils.py
    context_tokens = num_tokens_from_docs(top_docs) # this is a custom function we created in common/utils.py
    
    requested_tokens = prompt_tokens + context_tokens + COMPLETION_TOKENS
    
    chain_type = "map_reduce" if requested_tokens > 0.9 * tokens_limit else "stuff"  
    
    print("System prompt token count:",prompt_tokens)
    print("Max Completion Token count:", COMPLETION_TOKENS)
    print("Combined docs (context) token count:",context_tokens)
    print("--------")
    print("Requested token count:",requested_tokens)
    print("Token limit for", MODEL, ":", tokens_limit)
    print("Chain Type selected:", chain_type)
        
else:
    print("NO RESULTS FROM AZURE SEARCH")

System prompt token count: 271
Max Completion Token count: 1000
Combined docs (context) token count: 5231
--------
Requested token count: 6502
Token limit for gpt-35-turbo-16k : 16384
Chain Type selected: stuff


In [42]:
if chain_type == "stuff":
    chain = load_qa_with_sources_chain(llm, chain_type=chain_type, 
                                       prompt=COMBINE_PROMPT)
elif chain_type == "map_reduce":
    chain = load_qa_with_sources_chain(llm, chain_type=chain_type, 
                                       question_prompt=COMBINE_QUESTION_PROMPT,
                                       combine_prompt=COMBINE_PROMPT,
                                       return_intermediate_steps=True)

In [43]:
%%time
# Try with other language as well
response = chain({"input_documents": top_docs, "question": QUESTION, "language": "English"})

CPU times: user 69.4 ms, sys: 13.7 ms, total: 83.1 ms
Wall time: 1.25 s


In [44]:
display(Markdown(response['output_text']))

I'm sorry, but I couldn't find any information about the total number of robots in the provided extracts.